# Loading data with TimeBasedCesnetDataset

### Import

In [1]:
from tqdm import tqdm
import logging

from cesnet_tszoo.utils.enums import AgreggationType, SourceType, TimeFormat
from cesnet_tszoo.datasets import CESNET_TimeSeries24
from cesnet_tszoo.configs import TimeBasedConfig # Time based dataset MUST use TimeBasedConfig

### Setting logger

In [2]:
logging.basicConfig(
    level=logging.INFO,
    format="[%(asctime)s][%(name)s][%(levelname)s] - %(message)s")

### Preparing dataset

In [3]:
time_based_dataset = CESNET_TimeSeries24.get_dataset(data_root="/some_directory/", source_type=SourceType.INSTITUTION_SUBNETS, aggregation=AgreggationType.AGG_1_HOUR, is_series_based=False, display_details=True)

[2025-04-09 11:46:38,729][wrapper_dataset][INFO] - Dataset is time-based. Use cesnet_tszoo.configs.TimeBasedConfig



Dataset details:

    AgreggationType.AGG_1_HOUR
        Time indices: range(0, 6717)
        Datetime: (datetime.datetime(2023, 10, 9, 0, 0, tzinfo=datetime.timezone.utc), datetime.datetime(2024, 7, 14, 21, 0, tzinfo=datetime.timezone.utc))

    SourceType.INSTITUTION_SUBNETS
        Time series indices: [0 1 2 3 4 ... 543 544 545 546 547], Length=548; use 'get_available_ts_indices' for full list
        Features with default values: {'n_flows': 0, 'n_packets': 0, 'n_bytes': 0, 'tcp_udp_ratio_packets': 0.5, 'tcp_udp_ratio_bytes': 0.5, 'dir_ratio_packets': 0.5, 'dir_ratio_bytes': 0.5, 'avg_duration': 0, 'avg_ttl': 0, 'sum_n_dest_asn': 0, 'avg_n_dest_asn': 0, 'std_n_dest_asn': 0, 'sum_n_dest_ports': 0, 'avg_n_dest_ports': 0, 'std_n_dest_ports': 0, 'sum_n_dest_ip': 0, 'avg_n_dest_ip': 0, 'std_n_dest_ip': 0}
        
        Additional data: ['ids_relationship', 'weekends_and_holidays']
        


### Loading data with DataLoader

- Load data using Pytorch Dataloader.
- Last batch is never dropped (unless sliding_window is used).
- Workers affect how many processes will be used for loading data for specific set.
    - Workers set to 0, means loading will be ran on main process.
    - Set workers can be overriden in `get_*_dataloader` with parameter `workers`.
- Batch size affect how many times for every time series will be in one batch (differs when sliding window is used).
- Batch consists of: (only when sliding window is not used).
    - When `time_format` is not TimeFormat.DATETIME, then batch is one Numpy array of shape `(ts_ids/test_ts_ids, batch_size, features_to_take + used ids)`.
    - When `time_format` is TimeFormat.DATETIME, then batch is a tuple: (Numpy array of shape `(ts_ids/test_ts_ids, batch_size, features_to_take + used ids (without time))`, Numpy array of shape `(batch_size)`)

In [4]:
config = TimeBasedConfig(ts_ids=54, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, test_ts_ids=22, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128, all_batch_size=128)
time_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-04-09 11:46:38,737][config][INFO] - Quick validation succeeded.
[2025-04-09 11:46:38,758][config][INFO] - Finalization and validation completed successfully.
[2025-04-09 11:46:38,762][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time series.
100%|██████████| 54/54 [00:00<00:00, 557.25it/s]
[2025-04-09 11:46:38,870][cesnet_dataset][INFO] - Updating config on test_other and selected time series.
100%|██████████| 22/22 [00:00<00:00, 998.56it/s]
[2025-04-09 11:46:38,896][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Time series IDS: [258 264 243 329  30 ... 335 421 140 159  78], Length=54
        Test time series IDS: [198 247 144 547 313 ... 453  11 500 183 370], Length=22
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
        All time periods: range(0, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.5 0.5 0.  0. ]
        Time series ID incl

You can also change set batch sizes later with `update_dataset_config_and_initialize` or `set_batch_sizes`.

In [5]:
time_based_dataset.update_dataset_config_and_initialize(train_batch_size=33, val_batch_size=65, test_batch_size="config", all_batch_size="config")
# Or
time_based_dataset.set_batch_sizes(train_batch_size=33, val_batch_size=65, test_batch_size="config", all_batch_size="config")

[2025-04-09 11:46:38,902][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-04-09 11:46:38,903][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-04-09 11:46:38,904][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-04-09 11:46:38,904][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-04-09 11:46:38,905][cesnet_dataset][INFO] - Batch sizes has been changed successfuly.


You can also change set workers later with `update_dataset_config_and_initialize` or `set_workers`.

In [6]:
time_based_dataset.update_dataset_config_and_initialize(train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0)
# Or
time_based_dataset.set_workers(train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0)

[2025-04-09 11:46:38,911][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-04-09 11:46:38,911][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-04-09 11:46:38,913][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-04-09 11:46:38,913][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-04-09 11:46:38,913][cesnet_dataset][INFO] - Workers has been changed successfuly.


#### Train set

- Affected by `train_batch_size`.
- Affected by `train_workers`.

In [7]:
dataloader = time_based_dataset.get_train_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-04-09 11:46:38,922][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 102/102 [00:00<00:00, 190.10it/s]


(54, 33, 20)

#### Val set

- Affected by `val_batch_size`.
- Affected by `val_workers`.

In [8]:
dataloader = time_based_dataset.get_val_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-04-09 11:46:39,473][cesnet_dataset][INFO] - Created new cached val_dataloader.
100%|██████████| 31/31 [00:00<00:00, 127.64it/s]


(54, 65, 20)

#### Test set

- Affected by `test_batch_size`.
- Affected by `test_workers`.

In [9]:
dataloader = time_based_dataset.get_test_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-04-09 11:46:39,730][cesnet_dataset][INFO] - Created new cached test_dataloader.
100%|██████████| 11/11 [00:00<00:00, 99.92it/s]


(54, 128, 20)

##### When using test_ts_ids and test_time_period is set.

- Affected by `test_batch_size`.
- Affected by `test_workers`.

In [10]:
dataloader = time_based_dataset.get_test_other_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-04-09 11:46:39,853][cesnet_dataset][INFO] - Created new cached test_other_dataloader.
100%|██████████| 11/11 [00:00<00:00, 173.31it/s]


(22, 128, 20)

#### All set

- Affected by `all_batch_size`.
- Affected by `all_workers`.

In [11]:
dataloader = time_based_dataset.get_all_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-04-09 11:46:39,930][cesnet_dataset][INFO] - Created new cached all_dataloader.
100%|██████████| 53/53 [00:00<00:00, 164.67it/s]


(54, 128, 20)

#### Using time_format=TimeFormat.DATETIME

In [12]:
config = TimeBasedConfig(ts_ids=54, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, test_ts_ids=22, features_to_take="all", time_format=TimeFormat.DATETIME,
                         train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128, all_batch_size=128)
time_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-04-09 11:46:40,260][config][INFO] - Quick validation succeeded.
[2025-04-09 11:46:40,283][config][INFO] - Finalization and validation completed successfully.
[2025-04-09 11:46:40,289][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time series.
100%|██████████| 54/54 [00:00<00:00, 726.16it/s]
[2025-04-09 11:46:40,369][cesnet_dataset][INFO] - Updating config on test_other and selected time series.
100%|██████████| 22/22 [00:00<00:00, 1151.48it/s]
[2025-04-09 11:46:40,391][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Time series IDS: [433 466 415 511 327 ... 413  82 211  97 263], Length=54
        Test time series IDS: [403   2 296 531 441 ... 423 256 419 261 397], Length=22
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
        All time periods: range(0, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.5 0.5 0.  0. ]
        Time series ID incl

In [13]:
dataloader = time_based_dataset.get_train_dataloader(workers="config")

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0][0].shape) # data without time
display(batches[0][1].shape) # time

[2025-04-09 11:46:40,399][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 105/105 [00:00<00:00, 208.26it/s]


(54, 32, 19)

(32,)

#### Specifying which time series to load

- Every `get_*_dataloader` has parameter `ts_id`.
    - When `ts_id` is None, then it returns as previous examples.
    - When `ts_id` is not None, then it returns only one time series of that specified id.

In [14]:
config = TimeBasedConfig(ts_ids=[177, 176, 319, 267], train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, test_ts_ids=22, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128, all_batch_size=128)
time_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-04-09 11:46:40,913][config][INFO] - Quick validation succeeded.
[2025-04-09 11:46:40,932][config][INFO] - Finalization and validation completed successfully.
[2025-04-09 11:46:40,936][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time series.
100%|██████████| 4/4 [00:00<00:00, 726.41it/s]
[2025-04-09 11:46:40,947][cesnet_dataset][INFO] - Updating config on test_other and selected time series.
100%|██████████| 22/22 [00:00<00:00, 1816.61it/s]
[2025-04-09 11:46:40,961][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Time series IDS: [177 176 319 267], Length=4
        Test time series IDS: [277 113 245 482 546 ... 295 513 233 408  44], Length=22
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
        All time periods: range(0, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.5 0.5 0.  0. ]
        Time series ID included: True
        Time inclu

In [15]:
dataloader = time_based_dataset.get_train_dataloader(ts_id=177 , workers="config",)

batches = []

for batch in tqdm(dataloader):
    batches.append(batch)
    
display(batches[0].shape)

[2025-04-09 11:46:40,969][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 105/105 [00:00<00:00, 1895.71it/s]


(1, 32, 20)

#### Sliding window

- Both `sliding_window_size` and `sliding_window_prediction_size` must be set if you want to use sliding window.
- Batch sizes are used for background caching.
- Batch consists of:
    - When `time_format` is not TimeFormat.DATETIME, then batch is a tuple: <br>
    ( <br>
        Numpy array of shape `(ts_ids/test_ts_ids, sliding_window_size, features_to_take + used ids)`, <br>
        Numpy array of shape `(ts_ids/test_ts_ids, sliding_window_prediction_size, features_to_take + used ids)` <br>
    ).
    - When `time_format` is TimeFormat.DATETIME, then batch is a tuple: <br>
    ( <br>
        Numpy array of shape `(ts_ids/test_ts_ids, sliding_window_size, features_to_take + used ids (without time))`, <br>
        Numpy array of shape `(ts_ids/test_ts_ids, sliding_window_prediction_size, features_to_take + used ids (without time))`, <br>
        Numpy array of shape `(sliding_window_size)` of time, <br>
        Numpy array of shape `(sliding_window_prediction_size)` of time, <br>
    ).
- You can modify sliding window step size with `sliding_window_step`
- You can use `set_shared_size` to set how many times time periods should share.
    - `val_time_period` takes from `train_time_period`
    - `test_time_period` takes from `val_time_period` or `train_time_period`

In [16]:
config = TimeBasedConfig(ts_ids=54, train_time_period=range(0, 1000), val_time_period=range(1000, 1500), test_time_period=range(1500, 2000), test_ts_ids=22, features_to_take=["n_flows"], time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128, all_batch_size=128,
                         sliding_window_size=22, sliding_window_prediction_size=2, sliding_window_step=2, set_shared_size=0.05)
time_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-04-09 11:46:41,034][config][INFO] - Quick validation succeeded.
[2025-04-09 11:46:41,064][config][INFO] - Finalization and validation completed successfully.
[2025-04-09 11:46:41,068][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time series.
100%|██████████| 54/54 [00:00<00:00, 2342.44it/s]
[2025-04-09 11:46:41,096][cesnet_dataset][INFO] - Updating config on test_other and selected time series.
100%|██████████| 22/22 [00:00<00:00, 2094.53it/s]
[2025-04-09 11:46:41,109][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Time series IDS: [205  96 534  92 509 ... 518 144 146 528 274], Length=54
        Test time series IDS: [137 190 227 107  29 ... 462 539 250 249  97], Length=22
    Time periods
        Train time periods: range(0, 1000)
        Val time periods: range(665, 1500)
        Test time periods: range(1165, 2000)
        All time periods: range(0, 2000)
    Features
        Taken features: ['n_flows']
        Default values: [0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.ID_TIME
    Sliding window
        Sliding window size: 22
        Sliding window prediction size: 2
        Sliding window step size: 2
        Set shared size: 335
    Fillers
        Filler type: None
    Scalers
        Scaler type: None
    Batch sizes
        Train batch size: 32
        Val 

In [17]:
dataloader = time_based_dataset.get_train_dataloader(workers="config")

batches = []

for sliding_window, sliding_window_prediction in tqdm(dataloader):
    batches.append((sliding_window, sliding_window_prediction))

[2025-04-09 11:46:41,117][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 489/489 [00:00<00:00, 4528.79it/s]


You can also change sliding window parameters later with `update_dataset_config_and_initialize` or `set_sliding_window`.

In [18]:
time_based_dataset.update_dataset_config_and_initialize(sliding_window_size=22, sliding_window_prediction_size=3, sliding_window_step="config", set_shared_size="config", workers=0)
# Or
time_based_dataset.set_sliding_window(sliding_window_size=22, sliding_window_prediction_size=3, sliding_window_step="config", set_shared_size="config", workers=0)

[2025-04-09 11:46:41,231][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-04-09 11:46:41,232][cesnet_dataset][INFO] - Destroyed cached train_dataloader.
[2025-04-09 11:46:41,232][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-04-09 11:46:41,233][cesnet_dataset][INFO] - Re-initialization is not needed.
[2025-04-09 11:46:41,234][cesnet_dataset][INFO] - Configuration has been changed successfuly.
[2025-04-09 11:46:41,234][cesnet_dataset][INFO] - Sliding window values has been changed successfuly.


##### Using time_format=TimeFormat.DATETIME

In [19]:
config = TimeBasedConfig(ts_ids=54, train_time_period=range(0, 1000), val_time_period=range(978, 1500), test_time_period=range(1478, 2000), test_ts_ids=22, features_to_take=["n_flows"], time_format=TimeFormat.DATETIME,
                         train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0,
                         train_batch_size=32, val_batch_size=64, test_batch_size=128, all_batch_size=128,
                         sliding_window_size=22, sliding_window_prediction_size=2, sliding_window_step=2, set_shared_size=100)
time_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-04-09 11:46:41,240][config][INFO] - Quick validation succeeded.
[2025-04-09 11:46:41,259][config][INFO] - Finalization and validation completed successfully.
[2025-04-09 11:46:41,263][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time series.
100%|██████████| 54/54 [00:00<00:00, 2337.79it/s]
[2025-04-09 11:46:41,292][cesnet_dataset][INFO] - Updating config on test_other and selected time series.
100%|██████████| 22/22 [00:00<00:00, 2850.62it/s]
[2025-04-09 11:46:41,300][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Time series IDS: [297 532  37  58 352 ... 115 382 475 268 384], Length=54
        Test time series IDS: [483 223 153  93 247 ... 313 380 277 105 332], Length=22
    Time periods
        Train time periods: range(0, 1000)
        Val time periods: range(900, 1500)
        Test time periods: range(1400, 2000)
        All time periods: range(0, 2000)
    Features
        Taken features: ['n_flows']
        Default values: [0.]
        Time series ID included: True
        Time included: True    
        Time format: TimeFormat.DATETIME
    Sliding window
        Sliding window size: 22
        Sliding window prediction size: 2
        Sliding window step size: 2
        Set shared size: 100
    Fillers
        Filler type: None
    Scalers
        Scaler type: None
    Batch sizes
        Train batch size: 32
        Val

In [20]:
dataloader = time_based_dataset.get_train_dataloader(workers="config")

batches = []

for sliding_window, sliding_window_prediction, sliding_window_times, sliding_window_prediction_times in tqdm(dataloader):
    batches.append((sliding_window, sliding_window_prediction, sliding_window_times, sliding_window_prediction_times))

[2025-04-09 11:46:41,309][cesnet_dataset][INFO] - Created new cached train_dataloader.
100%|██████████| 489/489 [00:00<00:00, 4542.16it/s]


### Loading data as Dataframe

- Batch size has no effect.
- Sliding window has no effect.
- Returns every time series in `ts_ids`/`test_ts_ids` with sets specified time period.
- Data is returned as Pandas Dataframe.
- Workers affect how many processes will be used for loading data for specific set.
    - Workers set to 0, means loading will be ran on main process.
    - Set workers can be overriden in `get_*_df` with parameter `workers`.

In [21]:
config = TimeBasedConfig(ts_ids=54, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, test_ts_ids=22, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0)
time_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-04-09 11:46:41,423][config][INFO] - Quick validation succeeded.
[2025-04-09 11:46:41,442][config][INFO] - Finalization and validation completed successfully.
[2025-04-09 11:46:41,445][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time series.
100%|██████████| 54/54 [00:00<00:00, 703.22it/s]
[2025-04-09 11:46:41,528][cesnet_dataset][INFO] - Updating config on test_other and selected time series.
100%|██████████| 22/22 [00:00<00:00, 2313.81it/s]
[2025-04-09 11:46:41,539][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Time series IDS: [306 487 210 109 410 ... 409  49  66 282   5], Length=54
        Test time series IDS: [405 400 401 206  13 ... 136 188 352 313 384], Length=22
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
        All time periods: range(0, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.5 0.5 0.  0. ]
        Time series ID incl

#### Train set

- Affected by `train_workers`.

In [22]:
df = time_based_dataset.get_train_df(as_single_dataframe=True, workers="config")
dfs = time_based_dataset.get_train_df(as_single_dataframe=False, workers="config")

df.head(10)

,id_institution_subnet,id_time,n_flows,n_packets,n_bytes,sum_n_dest_asn,avg_n_dest_asn,std_n_dest_asn,sum_n_dest_ports,avg_n_dest_ports,std_n_dest_ports,sum_n_dest_ip,avg_n_dest_ip,std_n_dest_ip,tcp_udp_ratio_packets,tcp_udp_ratio_bytes,dir_ratio_packets,dir_ratio_bytes,avg_duration,avg_ttl
0,306.0,0.0,844.0,4368.0,665578.0,215.0,8.96,2.73,201.0,8.38,3.91,744.0,31.000000,12.540000,0.500000,0.549805,0.459961,0.429932,8.40,213.559998
1,306.0,1.0,1036.0,30018.0,7029939.0,248.0,10.33,3.05,215.0,8.96,3.80,901.0,37.540001,13.550000,0.489990,0.509766,0.489990,0.429932,8.64,208.059998
2,306.0,2.0,860.0,4523.0,1007375.0,189.0,7.88,2.03,174.0,7.25,2.67,775.0,32.290001,13.370000,0.419922,0.459961,0.479980,0.469971,11.23,214.009995
3,306.0,3.0,709.0,4014.0,774281.0,176.0,7.33,2.50,159.0,6.62,2.55,643.0,26.790001,9.810000,0.419922,0.439941,0.449951,0.429932,10.09,210.600006
4,306.0,4.0,679.0,3655.0,804308.0,178.0,7.42,2.95,132.0,5.50,2.81,630.0,26.250000,10.090000,0.379883,0.399902,0.459961,0.429932,9.76,209.429993
5,306.0,5.0,652.0,12294.0,6723946.0,178.0,7.42,4.55,121.0,5.04,2.22,597.0,24.879999,13.390000,0.459961,0.479980,0.489990,0.419922,10.16,207.240005
6,306.0,6.0,682.0,15403.0,9956893.0,205.0,8.54,5.03,128.0,5.33,1.79,633.0,26.379999,17.740000,0.449951,0.459961,0.439941,0.409912,12.01,205.750000
7,306.0,7.0,726.0,28460.0,25726918.0,210.0,8.75,6.10,135.0,5.62,2.20,636.0,26.500000,21.250000,0.489990,0.500000,0.439941,0.389893,9.56,194.229996
8,306.0,8.0,1120.0,126996.0,115255620.0,257.0,10.71,9.85,128.0,5.33,2.87,886.0,36.919998,34.990002,0.409912,0.419922,0.479980,0.419922,11.18,195.880005
9,306.0,9.0,1029.0,100004.0,64798009.0,232.0,9.67,6.66,161.0,6.71,2.63,871.0,36.290001,30.770000,0.529785,0.549805,0.429932,0.389893,11.59,198.580002


In [23]:
dfs

[      id_institution_subnet  id_time  n_flows  n_packets      n_bytes  \
 0                     306.0      0.0    844.0     4368.0     665578.0   
 1                     306.0      1.0   1036.0    30018.0    7029939.0   
 2                     306.0      2.0    860.0     4523.0    1007375.0   
 3                     306.0      3.0    709.0     4014.0     774281.0   
 4                     306.0      4.0    679.0     3655.0     804308.0   
 ...                     ...      ...      ...        ...          ...   
 3354                  306.0   3354.0    841.0   214752.0  226318815.0   
 3355                  306.0   3355.0    784.0   219830.0  231118828.0   
 3356                  306.0   3356.0    754.0    88345.0   89879917.0   
 3357                  306.0   3357.0    926.0    51670.0   51572476.0   
 3358                  306.0   3358.0    973.0   143682.0  149057819.0   
 
       sum_n_dest_asn  avg_n_dest_asn  std_n_dest_asn  sum_n_dest_ports  \
 0              215.0            8.

#### Val set

- Affected by `val_workers`.

In [24]:
df = time_based_dataset.get_val_df(as_single_dataframe=True, workers="config")
dfs = time_based_dataset.get_val_df(as_single_dataframe=False, workers="config")

df.head(10)

,id_institution_subnet,id_time,n_flows,n_packets,n_bytes,sum_n_dest_asn,avg_n_dest_asn,std_n_dest_asn,sum_n_dest_ports,avg_n_dest_ports,std_n_dest_ports,sum_n_dest_ip,avg_n_dest_ip,std_n_dest_ip,tcp_udp_ratio_packets,tcp_udp_ratio_bytes,dir_ratio_packets,dir_ratio_bytes,avg_duration,avg_ttl
0,306.0,3359.0,1109.0,125548.0,12203967.0,207.0,8.62,5.09,137.0,5.71,2.82,975.0,40.619999,16.48,0.330078,0.330078,0.509766,0.479980,18.40,212.919998
1,306.0,3360.0,830.0,6882.0,3241832.0,192.0,8.00,3.56,149.0,6.21,2.43,748.0,31.170000,14.78,0.419922,0.439941,0.459961,0.449951,13.61,206.080002
2,306.0,3361.0,873.0,292487.0,263838434.0,150.0,6.25,2.45,106.0,4.42,2.00,763.0,31.790001,14.67,0.350098,0.370117,0.509766,0.489990,13.80,209.820007
3,306.0,3362.0,968.0,3730.0,756057.0,165.0,6.88,2.63,132.0,5.50,2.21,858.0,35.750000,12.28,0.360107,0.389893,0.479980,0.469971,12.97,220.699997
4,306.0,3363.0,758.0,2670.0,583164.0,159.0,6.62,3.03,120.0,5.00,2.81,693.0,28.879999,8.39,0.300049,0.320068,0.509766,0.479980,14.74,219.020004
5,306.0,3364.0,642.0,3856.0,1280180.0,134.0,5.58,2.04,109.0,4.54,1.77,581.0,24.209999,10.14,0.399902,0.429932,0.479980,0.449951,13.71,212.770004
6,306.0,3365.0,628.0,7071.0,1121957.0,142.0,5.92,1.69,114.0,4.75,2.27,581.0,24.209999,6.98,0.360107,0.399902,0.469971,0.429932,12.00,217.699997
7,306.0,3366.0,611.0,3039.0,453543.0,121.0,5.04,2.22,85.0,3.54,1.61,565.0,23.540001,13.06,0.379883,0.409912,0.509766,0.489990,10.31,215.220001
8,306.0,3367.0,649.0,14794.0,8599800.0,149.0,6.21,2.52,95.0,3.96,1.71,600.0,25.000000,11.35,0.350098,0.370117,0.459961,0.429932,12.97,206.910004
9,306.0,3368.0,586.0,13944.0,3087170.0,136.0,5.67,3.55,85.0,3.54,1.79,543.0,22.620001,14.48,0.320068,0.330078,0.500000,0.469971,13.45,212.660004


In [25]:
dfs

[      id_institution_subnet  id_time  n_flows  n_packets      n_bytes  \
 0                     306.0   3359.0   1109.0   125548.0   12203967.0   
 1                     306.0   3360.0    830.0     6882.0    3241832.0   
 2                     306.0   3361.0    873.0   292487.0  263838434.0   
 3                     306.0   3362.0    968.0     3730.0     756057.0   
 4                     306.0   3363.0    758.0     2670.0     583164.0   
 ...                     ...      ...      ...        ...          ...   
 2010                  306.0   5369.0    592.0    45178.0   13315289.0   
 2011                  306.0   5370.0    693.0     2975.0     781410.0   
 2012                  306.0   5371.0    577.0     1769.0     268869.0   
 2013                  306.0   5372.0    727.0     2415.0     403545.0   
 2014                  306.0   5373.0    701.0     3107.0     618131.0   
 
       sum_n_dest_asn  avg_n_dest_asn  std_n_dest_asn  sum_n_dest_ports  \
 0              207.0            8.

#### Test set

- Affected by `test_workers`.

In [26]:
df = time_based_dataset.get_test_df(as_single_dataframe=True, workers="config")
dfs = time_based_dataset.get_test_df(as_single_dataframe=False, workers="config")

df.head(10)

,id_institution_subnet,id_time,n_flows,n_packets,n_bytes,sum_n_dest_asn,avg_n_dest_asn,std_n_dest_asn,sum_n_dest_ports,avg_n_dest_ports,std_n_dest_ports,sum_n_dest_ip,avg_n_dest_ip,std_n_dest_ip,tcp_udp_ratio_packets,tcp_udp_ratio_bytes,dir_ratio_packets,dir_ratio_bytes,avg_duration,avg_ttl
0,306.0,5374.0,665.0,1791.0,201497.0,130.0,5.42,2.90,103.0,4.29,2.03,598.0,24.920000,10.050000,0.290039,0.340088,0.459961,0.439941,12.62,216.850006
1,306.0,5375.0,681.0,1941.0,217400.0,134.0,5.83,2.50,117.0,5.09,2.52,614.0,26.700001,9.580000,0.340088,0.389893,0.449951,0.399902,9.70,220.039993
2,306.0,5376.0,624.0,1720.0,223405.0,136.0,5.67,1.71,121.0,5.04,1.88,562.0,23.420000,8.400000,0.379883,0.409912,0.479980,0.439941,12.07,211.179993
3,306.0,5377.0,533.0,1861.0,352011.0,122.0,5.08,2.21,93.0,3.88,1.87,480.0,20.000000,12.170000,0.340088,0.370117,0.500000,0.479980,9.23,215.649994
4,306.0,5378.0,658.0,62648.0,65362845.0,142.0,5.92,2.30,126.0,5.25,2.61,585.0,24.379999,12.660000,0.389893,0.419922,0.419922,0.370117,14.20,209.339996
5,306.0,5379.0,531.0,1819.0,298559.0,129.0,5.38,2.75,100.0,4.17,1.76,478.0,19.920000,10.630000,0.360107,0.379883,0.469971,0.429932,8.86,214.669998
6,306.0,5380.0,598.0,21745.0,23691799.0,123.0,5.12,2.98,97.0,4.04,2.27,532.0,22.170000,13.160000,0.350098,0.360107,0.409912,0.360107,8.69,202.350006
7,306.0,5381.0,557.0,7903.0,5766050.0,149.0,6.21,6.41,85.0,3.54,2.19,495.0,20.620001,17.070000,0.330078,0.350098,0.469971,0.429932,9.86,204.710007
8,306.0,5382.0,698.0,286298.0,315770519.0,147.0,6.12,5.11,101.0,4.21,2.38,609.0,25.379999,19.700001,0.409912,0.419922,0.399902,0.360107,14.63,200.770004
9,306.0,5383.0,938.0,160325.0,172206956.0,190.0,7.92,8.85,90.0,3.75,2.38,783.0,32.619999,31.990000,0.330078,0.340088,0.449951,0.419922,13.24,206.820007


In [27]:
dfs

[      id_institution_subnet  id_time  n_flows  n_packets      n_bytes  \
 0                     306.0   5374.0    665.0     1791.0     201497.0   
 1                     306.0   5375.0    681.0     1941.0     217400.0   
 2                     306.0   5376.0    624.0     1720.0     223405.0   
 3                     306.0   5377.0    533.0     1861.0     352011.0   
 4                     306.0   5378.0    658.0    62648.0   65362845.0   
 ...                     ...      ...      ...        ...          ...   
 1338                  306.0   6712.0   1597.0   265332.0  157152918.0   
 1339                  306.0   6713.0   1513.0   506105.0  428821560.0   
 1340                  306.0   6714.0   1413.0   315057.0  231505212.0   
 1341                  306.0   6715.0   1672.0   276917.0  240058433.0   
 1342                  306.0   6716.0   1227.0   508628.0  431543980.0   
 
       sum_n_dest_asn  avg_n_dest_asn  std_n_dest_asn  sum_n_dest_ports  \
 0              130.0            5.

##### When using test_ts_ids and test_time_period is set.

- Affected by `test_workers`.

In [28]:
df = time_based_dataset.get_test_other_df(as_single_dataframe=True, workers="config")
dfs = time_based_dataset.get_test_other_df(as_single_dataframe=False, workers="config")

df.head(10)

,id_institution_subnet,id_time,n_flows,n_packets,n_bytes,sum_n_dest_asn,avg_n_dest_asn,std_n_dest_asn,sum_n_dest_ports,avg_n_dest_ports,std_n_dest_ports,sum_n_dest_ip,avg_n_dest_ip,std_n_dest_ip,tcp_udp_ratio_packets,tcp_udp_ratio_bytes,dir_ratio_packets,dir_ratio_bytes,avg_duration,avg_ttl
0,405.0,5374.0,6472.0,21220.0,6179766.0,505.0,7.11,4.87,747.0,10.52,12.21,1195.0,16.830000,19.240000,0.770020,0.799805,0.360107,0.320068,21.030001,115.150002
1,405.0,5375.0,4532.0,23099.0,6028167.0,493.0,7.36,5.35,919.0,13.72,28.40,1140.0,17.010000,20.049999,0.790039,0.839844,0.439941,0.379883,20.090000,112.610001
2,405.0,5376.0,1730.0,47895.0,29175998.0,442.0,7.02,4.52,377.0,5.98,2.99,1117.0,17.730000,20.299999,0.740234,0.790039,0.459961,0.409912,22.080000,123.809998
3,405.0,5377.0,1869.0,20564.0,9086592.0,509.0,7.17,5.11,408.0,5.75,3.27,1162.0,16.370001,18.450001,0.779785,0.819824,0.479980,0.409912,25.200001,122.230003
4,405.0,5378.0,1912.0,25911.0,12912663.0,474.0,6.97,6.21,399.0,5.87,4.55,1152.0,16.940001,27.959999,0.819824,0.859863,0.500000,0.469971,21.650000,119.309998
5,405.0,5379.0,3307.0,252523.0,269931460.0,649.0,9.27,12.12,395.0,5.64,2.67,1953.0,27.900000,58.900002,0.859863,0.899902,0.479980,0.399902,25.469999,113.290001
6,405.0,5380.0,4932.0,251909.0,251500870.0,742.0,10.16,11.48,560.0,7.67,12.51,2534.0,34.709999,72.050003,0.799805,0.819824,0.449951,0.370117,27.750000,118.470001
7,405.0,5381.0,7003.0,337685.0,343821954.0,878.0,12.03,14.44,601.0,8.23,11.06,3085.0,42.259998,84.279999,0.839844,0.859863,0.439941,0.340088,35.230000,108.000000
8,405.0,5382.0,5827.0,253193.0,254258917.0,873.0,12.12,14.54,457.0,6.35,2.77,3127.0,43.430000,87.980003,0.819824,0.850098,0.479980,0.389893,32.119999,114.820000
9,405.0,5383.0,5413.0,230053.0,194695246.0,840.0,11.83,12.93,675.0,9.51,11.49,2779.0,39.139999,75.540001,0.859863,0.899902,0.449951,0.370117,38.630001,100.989998


In [29]:
dfs

[      id_institution_subnet  id_time  n_flows  n_packets     n_bytes  \
 0                     405.0   5374.0   6472.0    21220.0   6179766.0   
 1                     405.0   5375.0   4532.0    23099.0   6028167.0   
 2                     405.0   5376.0   1730.0    47895.0  29175998.0   
 3                     405.0   5377.0   1869.0    20564.0   9086592.0   
 4                     405.0   5378.0   1912.0    25911.0  12912663.0   
 ...                     ...      ...      ...        ...         ...   
 1338                  405.0   6712.0   5683.0    32162.0  12020386.0   
 1339                  405.0   6713.0   6104.0    70754.0  53692852.0   
 1340                  405.0   6714.0   6323.0    99040.0  66047243.0   
 1341                  405.0   6715.0   6318.0    28522.0   8438158.0   
 1342                  405.0   6716.0   6298.0    63664.0  10893854.0   
 
       sum_n_dest_asn  avg_n_dest_asn  std_n_dest_asn  sum_n_dest_ports  \
 0              505.0            7.11          

#### All set

- Affected by `all_workers`.

In [30]:
df = time_based_dataset.get_all_df(as_single_dataframe=True, workers="config")
dfs = time_based_dataset.get_all_df(as_single_dataframe=False, workers="config")

df.head(10)

,id_institution_subnet,id_time,n_flows,n_packets,n_bytes,sum_n_dest_asn,avg_n_dest_asn,std_n_dest_asn,sum_n_dest_ports,avg_n_dest_ports,std_n_dest_ports,sum_n_dest_ip,avg_n_dest_ip,std_n_dest_ip,tcp_udp_ratio_packets,tcp_udp_ratio_bytes,dir_ratio_packets,dir_ratio_bytes,avg_duration,avg_ttl
0,306.0,0.0,844.0,4368.0,665578.0,215.0,8.96,2.73,201.0,8.38,3.91,744.0,31.000000,12.540000,0.500000,0.549805,0.459961,0.429932,8.40,213.559998
1,306.0,1.0,1036.0,30018.0,7029939.0,248.0,10.33,3.05,215.0,8.96,3.80,901.0,37.540001,13.550000,0.489990,0.509766,0.489990,0.429932,8.64,208.059998
2,306.0,2.0,860.0,4523.0,1007375.0,189.0,7.88,2.03,174.0,7.25,2.67,775.0,32.290001,13.370000,0.419922,0.459961,0.479980,0.469971,11.23,214.009995
3,306.0,3.0,709.0,4014.0,774281.0,176.0,7.33,2.50,159.0,6.62,2.55,643.0,26.790001,9.810000,0.419922,0.439941,0.449951,0.429932,10.09,210.600006
4,306.0,4.0,679.0,3655.0,804308.0,178.0,7.42,2.95,132.0,5.50,2.81,630.0,26.250000,10.090000,0.379883,0.399902,0.459961,0.429932,9.76,209.429993
5,306.0,5.0,652.0,12294.0,6723946.0,178.0,7.42,4.55,121.0,5.04,2.22,597.0,24.879999,13.390000,0.459961,0.479980,0.489990,0.419922,10.16,207.240005
6,306.0,6.0,682.0,15403.0,9956893.0,205.0,8.54,5.03,128.0,5.33,1.79,633.0,26.379999,17.740000,0.449951,0.459961,0.439941,0.409912,12.01,205.750000
7,306.0,7.0,726.0,28460.0,25726918.0,210.0,8.75,6.10,135.0,5.62,2.20,636.0,26.500000,21.250000,0.489990,0.500000,0.439941,0.389893,9.56,194.229996
8,306.0,8.0,1120.0,126996.0,115255620.0,257.0,10.71,9.85,128.0,5.33,2.87,886.0,36.919998,34.990002,0.409912,0.419922,0.479980,0.419922,11.18,195.880005
9,306.0,9.0,1029.0,100004.0,64798009.0,232.0,9.67,6.66,161.0,6.71,2.63,871.0,36.290001,30.770000,0.529785,0.549805,0.429932,0.389893,11.59,198.580002


In [31]:
dfs

[      id_institution_subnet  id_time  n_flows  n_packets      n_bytes  \
 0                     306.0      0.0    844.0     4368.0     665578.0   
 1                     306.0      1.0   1036.0    30018.0    7029939.0   
 2                     306.0      2.0    860.0     4523.0    1007375.0   
 3                     306.0      3.0    709.0     4014.0     774281.0   
 4                     306.0      4.0    679.0     3655.0     804308.0   
 ...                     ...      ...      ...        ...          ...   
 6712                  306.0   6712.0   1597.0   265332.0  157152918.0   
 6713                  306.0   6713.0   1513.0   506105.0  428821560.0   
 6714                  306.0   6714.0   1413.0   315057.0  231505212.0   
 6715                  306.0   6715.0   1672.0   276917.0  240058433.0   
 6716                  306.0   6716.0   1227.0   508628.0  431543980.0   
 
       sum_n_dest_asn  avg_n_dest_asn  std_n_dest_asn  sum_n_dest_ports  \
 0              215.0            8.

### Loading data as singular Numpy array 

- Batch size has no effect.
- Sliding window has no effect.
- Returns every time series in `ts_ids`/`test_ts_ids` with sets specified time period.
- Data is returned as one Numpy array.
- Follows similar rules to Dataloader batches, regarding shape (excluding sliding window parameters).
- Workers affect how many processes will be used for loading data for specific set.
    - Workers set to 0, means loading will be ran on main process.
    - Set workers can be overriden in `get_*_numpy` with parameter `workers`.

In [32]:
config = TimeBasedConfig(ts_ids=54, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, test_ts_ids=22, features_to_take="all", time_format=TimeFormat.ID_TIME,
                         train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0)
time_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-04-09 11:46:43,291][config][INFO] - Quick validation succeeded.
[2025-04-09 11:46:43,310][config][INFO] - Finalization and validation completed successfully.
[2025-04-09 11:46:43,315][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time series.
100%|██████████| 54/54 [00:00<00:00, 722.00it/s]
[2025-04-09 11:46:43,395][cesnet_dataset][INFO] - Updating config on test_other and selected time series.
100%|██████████| 22/22 [00:00<00:00, 2314.74it/s]
[2025-04-09 11:46:43,407][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Time series IDS: [535 133 282  58 281 ... 227 501 122 173 142], Length=54
        Test time series IDS: [547 234 151   4 214 ... 504  21 472 192 285], Length=22
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
        All time periods: range(0, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.5 0.5 0.  0. ]
        Time series ID incl

#### Train set

- Affected by `train_workers`.

In [33]:
numpy_array = time_based_dataset.get_train_numpy(workers="config")

display(numpy_array.shape)

(54, 3359, 20)

#### Val set

- Affected by `val_workers`.

In [34]:
numpy_array = time_based_dataset.get_val_numpy(workers="config")

display(numpy_array.shape)

(54, 2015, 20)

#### Test set

- Affected by `test_workers`.

In [35]:
numpy_array = time_based_dataset.get_test_numpy(workers="config")

display(numpy_array.shape)

(54, 1343, 20)

##### When using test_ts_ids and test_time_period is set.

- Affected by `test_workers`.

In [36]:
numpy_array = time_based_dataset.get_test_other_numpy(workers="config")

display(numpy_array.shape)

(22, 1343, 20)

#### All set

- Affected by `all_workers`.

In [37]:
numpy_array = time_based_dataset.get_all_numpy(workers="config")

display(numpy_array.shape)

(54, 6717, 20)

#### Using time_format=TimeFormat.DATETIME

In [38]:
config = TimeBasedConfig(ts_ids=54, train_time_period=0.5, val_time_period=0.3, test_time_period=0.2, test_ts_ids=22, features_to_take="all", time_format=TimeFormat.DATETIME,
                         train_workers=0, val_workers=0, test_workers=0, all_workers=0, init_workers=0)
time_based_dataset.set_dataset_config_and_initialize(config, display_config_details=True, workers=0)

[2025-04-09 11:46:43,730][config][INFO] - Quick validation succeeded.
[2025-04-09 11:46:43,754][config][INFO] - Finalization and validation completed successfully.
[2025-04-09 11:46:43,757][cesnet_dataset][INFO] - Updating config on train/val/test/all and selected time series.
100%|██████████| 54/54 [00:00<00:00, 1296.01it/s]
[2025-04-09 11:46:43,803][cesnet_dataset][INFO] - Updating config on test_other and selected time series.
100%|██████████| 22/22 [00:00<00:00, 2403.24it/s]
[2025-04-09 11:46:43,814][cesnet_dataset][INFO] - Config initialized successfully.



Config Details
    Used for database: CESNET-TimeSeries24
    Aggregation: AgreggationType.AGG_1_HOUR
    Source: SourceType.INSTITUTION_SUBNETS

    Time series
        Time series IDS: [486 169 387 136 174 ... 162 499 244 119 336], Length=54
        Test time series IDS: [457 194 151 462 241 ... 403 367 410  78 102], Length=22
    Time periods
        Train time periods: range(0, 3359)
        Val time periods: range(3359, 5374)
        Test time periods: range(5374, 6717)
        All time periods: range(0, 6717)
    Features
        Taken features: ['n_flows', 'n_packets', 'n_bytes', 'sum_n_dest_asn', 'avg_n_dest_asn', 'std_n_dest_asn', 'sum_n_dest_ports', 'avg_n_dest_ports', 'std_n_dest_ports', 'sum_n_dest_ip', 'avg_n_dest_ip', 'std_n_dest_ip', 'tcp_udp_ratio_packets', 'tcp_udp_ratio_bytes', 'dir_ratio_packets', 'dir_ratio_bytes', 'avg_duration', 'avg_ttl']
        Default values: [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.5 0.5 0.5 0.5 0.  0. ]
        Time series ID incl

In [39]:
numpy_array, times = time_based_dataset.get_train_numpy(workers="config")

display(numpy_array.shape)
display(times)

(54, 3359, 19)

array([datetime.datetime(2023, 10, 9, 0, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2023, 10, 9, 1, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2023, 10, 9, 2, 0, tzinfo=datetime.timezone.utc),
       ...,
       datetime.datetime(2024, 2, 25, 20, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2024, 2, 25, 21, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2024, 2, 25, 22, 0, tzinfo=datetime.timezone.utc)],
      dtype=object)